In [ ]:
from skimage.data import imread
from skimage.io import imshow,imsave
from skimage import img_as_float
import pandas as pd
import numpy as np
import cv2
from skimage.util import crop
from skimage.transform import rotate
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [ ]:
def deg_angle_between(x1,y1,x2,y2):
    from math import atan2, degrees, pi
    dx = x2 - x1
    dy = y2 - y1
    rads = atan2(-dy,dx)
    rads %= 2*pi
    degs = degrees(rads)
    return(degs)

In [ ]:
def get_rotated_cropped_fish(img,x1,y1,x2,y2):
    (h,w) = img.shape[:2]
    #calculate center and angle
    center = ( (x1+x2) / 2,(y1+y2) / 2)
    angle = np.floor(-deg_angle_between(x1,y1,x2,y2))
    #print('angle=' +str(angle) + ' ')
    #print('center=' +str(center))
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    
    fish_length = np.sqrt((x1-x2)**2+(y1-y2)**2)
    cropped = rotated[(max((center[1]-fish_length/1.8),0)):(max((center[1]+fish_length/1.8),0)) ,
                      (max((center[0]- fish_length/1.8),0)):(max((center[0]+fish_length/1.8),0))]
    #imshow(img)
    #imshow(rotated)
    #imshow(cropped)
    resized = resize(cropped,(224,224))
    return(resized)

In [ ]:
label_files = ['C:/Users/Mohan/Downloads/bet_labels.json',
             'C:/Users/Mohan/Downloads/alb_labels.json',
             'C:/Users/Mohan/Downloads/yft_labels.json',
             'C:/Users/Mohan/Downloads/dol_labels.json',
             'C:/Users/Mohan/Downloads/shark_labels.json',
             'C:/Users/Mohan/Downloads/lag_labels.json',
             'C:/Users/Mohan/Downloads/other_labels.json']

data_dirs = ['C:/Users/Mohan/Downloads/train/BET/',
             'C:/Users/Mohan/Downloads/train/ALB/',
             'C:/Users/Mohan/Downloads/train/YFT/',
             'C:/Users/Mohan/Downloads/train/DOL/',
             'C:/Users/Mohan/Downloads/train/SHARK/',
             'C:/Users/Mohan/Downloads/train/LAG/',
             'C:/Users/Mohan/Downloads/train/OTHER/']


In [ ]:
images = list()
labels_list = list()
for c in range(7):
    labels = pd.read_json(label_files[c])
    for i in range(len(labels)):
        try:
            img_filename = labels.iloc[i,2]
            print(img_filename)
            l1 = pd.DataFrame((labels[labels.filename==img_filename].annotations).iloc[0])
            image = imread(data_dirs[c]+img_filename)
            images.append(get_rotated_cropped_fish(image,np.floor(l1.iloc[0,1]),np.floor(l1.iloc[0,2]),np.floor(l1.iloc[1,1]),np.floor(l1.iloc[1,2])))
            print('success')
            labels_list.append(c)
        except:
            pass

In [ ]:
labels =pd.read_json(label_files[0])
img_filename = labels.iloc[1,2]
print(img_filename)
l1 = pd.DataFrame((labels[labels.filename==img_filename].annotations).iloc[0])
image = imread(data_dirs[0]+img_filename)
images.append(get_rotated_cropped_fish(image,np.floor(l1.iloc[0,1]),np.floor(l1.iloc[0,2]),np.floor(l1.iloc[1,1]),np.floor(l1.iloc[1,2])))
print('success')            

In [ ]:
for i in range(len(images)):
    imsave('C:/Users/Mohan/Downloads/train/cropped/ALB/img_'+str(i)+'label_'+str(labels_list[i])+'.jpg',images[i])

In [ ]:
#for i in range(50):
#    fig,ax = plt.subplots(nrows=1,ncols=8,sharex="col",sharey="row",figsize=(24,3))
#    fig.suptitle(str(labels_list[(i*8):(8+i*8)]),fontsize=16)
#    for j in range(8):
#        ax[j].imshow(images[j+i*5])